<h2>Для работы нужно поднять сервис на гугле с SpeechToText и TextToSpeech API и скачать его credentials. Также поставить PyAudio, Playsound и SymPy. Поддерживается только русский, говорить запросы вроде:<h2>
    
1. 10 плюс 2  
2. 10 умножить на 2  
3. икс плюс 3 равно 10  
4. и т.д.  

In [0]:
from google.cloud import texttospeech
from google.cloud import speech
from google.oauth2 import service_account

import pyaudio
from playsound import playsound
from sympy.parsing.sympy_parser import parse_expr, standard_transformations, implicit_multiplication_application
from sympy.abc import x
from sympy import S, Eq, E
from sympy.solvers import solveset

In [0]:
RATE = 22050

In [0]:
def record_voice(player, seconds=4):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RECORD_SECONDS = seconds

    stream = player.open(
                input=True,
                format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                frames_per_buffer=CHUNK)

    print("Start recording... ", end="")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("recorded", seconds, "second(s)")
    stream.stop_stream()
    stream.close()

    return frames

In [0]:
def get_response_text(responses):
    #достаем распознанный гуглом текст из ответа
    transcripts = []
    for response in responses:
        if not response.results:
            continue

        result = response.results[0]

        if not result.alternatives:
            continue

        transcripts.append(result.alternatives[0].transcript)
    return transcripts[-1]

In [0]:
#настраиваем клиент по доке гугла, записываем и отправляем ауидо в SpeechToText и получаем транскрипт записи
def speech_to_text(client, language_code, player):
    config = speech.types.RecognitionConfig(
        encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code)
    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True)
    
    frames = record_voice(player)
    
    requests = (types.StreamingRecognizeRequest(audio_content=content)
                for content in frames)

    responses = client.streaming_recognize(streaming_config, requests)
    return get_response_text(responses)

In [0]:
# вычисляем наш экспрешион
def calculate(expr) :
    # парсим экспершион в нормальный вид
    def preprocess_equation():
        parse_dict = {'икс':'x', 'х':'x', 'плюс':'+', 'минус':'-', 'умножить':'*', 'на':'', 'делить':'/'}
        left_part = ""
        right_part = ""
        isEquation = False
        for word in expr.lower().split(' '):
            # нашли равно - значит это уравнение (часть 2 таски)
            if word == 'равно':
                isEquation = True
            # парсим правую часть, чтобы потом запихнуть в solveset
            elif isEquation:
                if (word in parse_dict):
                    right_part = right_part + parse_dict[word]
                else:
                    right_part = right_part + word
            # парсим левую часть
            else:
                if (word in parse_dict):
                    left_part = left_part + parse_dict[word]
                else:
                    left_part = left_part + word
                    
        #гугл сервис пишет умножить как 'x', поэтому если это численное выражение, то меняем 'x'на '*'
        if not isEquation and 'x' in left_part:
            left_part = left_part.replace('x', '*')
        return (left_part, right_part)
    
    (eq, answer) = preprocess_equation() 
    # если уравнение, то парсим его части из строк в SymPy expression
    transformations = (standard_transformations + (implicit_multiplication_application,))
    if answer:
        # чтобы x5 парсилось в x*5
        left = parse_expr(eq, transformations=transformations, evaluate=True)
        right = parse_expr(answer, transformations=transformations, evaluate=True)
        x = sympy.symbols('x')
        # решаем уравнение и достаем ответ из FiniteSet
        return solveset(Eq(left, right), x, domain=S.Reals).args[0]
    else:
        # решаем обычный numerical expression
        return parse_expr(eq, transformations=transformations, evaluate=True)

In [0]:
# настраиваем клиент TextToSpeech, кидаем результат вычислений в сервис, получаем и воспроизводим mp3
def text_to_speech(client, text, language, player):
    import base64
    synthesis_input = texttospeech.types.SynthesisInput(text=text)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code=language,
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    
    with open('out.mp3', 'wb') as speech:
        speech.write(response.audio_content)
    playsound('out.mp3')

In [0]:
def main():
    language = "ru-RU"
    # создание SpeechToText и TextToSpeech клиентов
    credentials = service_account.Credentials.from_service_account_file("/home/kn/practicalAI/exam/googleSpeech.json")
    clientSpeechToText = speech.SpeechClient(credentials=credentials)
    clientTextToSpeech = texttospeech.TextToSpeechClient(credentials=credentials)
    player = pyaudio.PyAudio()
    
    text = speech_to_text(clientSpeechToText, language, player)
    print(text)
    result = calculate(text)
    print(result)
    text_to_speech(clientTextToSpeech, str(result), language, player)
    
if __name__ == '__main__':
    main()

Start recording... recorded 4 second(s)
5 - x равно 8
-3
